In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [3]:
source = 'source/AIDSinfo_2014_en.csv'

In [9]:
data = pd.read_csv(source, error_bad_lines=False)

b'Skipping line 272: expected 9 fields, saw 10\n'
b'Skipping line 274079: expected 9 fields, saw 10\n'


In [20]:
data = pd.read_csv(source, error_bad_lines=False)

In [22]:
data.head()

,Indicator,Unit,Subgroup,Area,Area ID,Time Period,Source,Data Value,Footnotes
0,Laws that impose compulsory detention for peop...,Yes/No,Reported,Viet Nam,VNM,2014,UNAIDS_GARPR_,1.0,NaN
1,Laws that impose compulsory detention for peop...,Yes/No,Reported,Sri Lanka,LKA,2014,UNAIDS_GARPR_,1.0,NaN
2,Laws that impose compulsory detention for peop...,Yes/No,Reported,Brunei Darussalam,BRN,2014,UNAIDS_GARPR_,1.0,NaN
3,Laws that impose compulsory detention for peop...,Yes/No,Reported,Indonesia,IDN,2014,UNAIDS_GARPR_,1.0,NaN
4,Laws that impose compulsory detention for peop...,Yes/No,Reported,Singapore,SGP,2014,UNAIDS_GARPR_,1.0,NaN


In [23]:
data['Indicator'] = data['Indicator'].str.strip()

In [84]:
idx1 = data.query("Subgroup == '25+'").index
idx2 = data.query("Subgroup == '< 25'").index

In [91]:
data.loc[idx1, 'Subgroup'] = '25plus'
data.loc[idx2, 'Subgroup'] = 'below25'

In [92]:
data['cname'] = data['Indicator'] + ' ' + data['Subgroup']

In [94]:
data['cname'].ix[0]

'Laws that impose compulsory detention for people who use drugs Reported'

In [95]:
conc = data[['cname', 'Unit']].copy()

In [96]:
conc = conc.drop_duplicates()

In [98]:
conc.columns = ['name', 'unit']

In [99]:
conc['concept_type'] = 'measure'

In [100]:
conc['concept'] = conc['name'].map(lambda x: to_concept_id(x, '[/ -\\.\\*";:]+'))

In [101]:
np.max(conc['concept'].map(len))

124

In [102]:
conc.head()

,name,unit,concept_type,concept
0,Laws that impose compulsory detention for peop...,Yes/No,measure,laws_that_impose_compulsory_detention_for_peop...
5,Laws that specifically criminalise HIV non-dis...,Yes/No,measure,laws_that_specifically_criminalise_hiv_non_dis...
8,External economic support to the poorest house...,Percent,measure,external_economic_support_to_the_poorest_house...
22,Sex workers: Condom Use Females,Percent,measure,sex_workers_condom_use_females
27,Laws that provide for death penalty for drug o...,Yes/No,measure,laws_that_provide_for_death_penalty_for_drug_o...


In [103]:
disc = pd.DataFrame([['Name', np.nan, 'string', 'name'],
                     ['Year', 'year', 'time', 'year'],
                     ['Area', np.nan, 'entity_domain', 'area'],
                     ['unit', np.nan, 'string', 'Unit']
                    ], columns=conc.columns)

In [104]:
concept = pd.concat([disc, conc])

In [105]:
concept[['concept', 'name', 'concept_type', 'unit']].to_csv('../ddf--concepts.csv', index=False)

In [64]:
concept.iloc[100, 3]

'hiv_in_new_tb_cases'

In [45]:
area = data[['Area ID', 'Area']]

In [47]:
area.columns = ['area', 'name']

In [49]:
area.drop_duplicates().sort_values(by='name').to_csv('../ddf--entities--country.csv', index=False)

In [106]:
dps = data[['cname', 'Time Period', 'Area ID', 'Data Value']].copy()

In [107]:
dps.columns = ['concept', 'year', 'area', 'data']

In [108]:
dps['concept'] = dps['concept'].map(lambda x: to_concept_id(x, '[/ -\\.\\*";:]+'))

In [109]:
dps_gps = dps.groupby(by='concept')

In [113]:
for k, idx in dps_gps.groups.items():
    df_ = dps.ix[idx][['year', 'area', 'data']].copy()
    
    df_.columns = ['year','area', k]
    
    path = os.path.join('../', 'ddf--datapoints--{}--by--area--year.csv'.format(k))
    
    assert(np.all(df_[['year', 'area']].duplicated()) == False)
    
    df_.sort_values(by=['area', 'year']).to_csv(path, index=False)

In [115]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
2,concept,unit,ddf--concepts.csv
0,"area,year",a_i_1_2_women_sector_has_earmarked_budget_repo...,ddf--datapoints--a_i_1_2_women_sector_has_earm...
0,"area,year",a_i_1_2_women_sector_included_in_strategy_repo...,ddf--datapoints--a_i_1_2_women_sector_included...
0,"area,year",a_i_1_3_strategy_addresses_gender_empowerment_...,ddf--datapoints--a_i_1_3_strategy_addresses_ge...
0,"area,year",a_i_1_3_strategy_addresses_human_rights_protec...,ddf--datapoints--a_i_1_3_strategy_addresses_hu...
0,"area,year",a_i_1_3_strategy_addresses_people_who_inject_d...,ddf--datapoints--a_i_1_3_strategy_addresses_pe...
0,"area,year",a_i_1_3_strategy_addresses_stigma_and_discrimi...,ddf--datapoints--a_i_1_3_strategy_addresses_st...
0,"area,year",a_i_1_3_strategy_addresses_transgender_people_...,ddf--datapoints--a_i_1_3_strategy_addresses_tr...
